# Chapter 15: Self-Attention and Transformers
The Transformer Encoder and its implementation in PyTorch

Programs from the book: [_Python for Natural Language Processing_](https://link.springer.com/book/9783031575488)

__Author__: Pierre Nugues

## Modules

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import copy

In [2]:
random.seed(1234)
torch.manual_seed(1234)

## Attention

In [3]:
def read_embeddings(file):
    embeddings = {}
    with open(file, encoding='utf8') as glove:
        for line in glove:
            values = line.strip().split()
            word = values[0]
            embeddings[word] = torch.FloatTensor(
                [float(value) for value in values[1:]])
    return embeddings

In [4]:
PATH = '../datasets/embeddings/'

In [5]:
embedding_file = PATH + 'glove.6B.50d.txt'
embeddings_dict = read_embeddings(embedding_file)

In [6]:
# embeddings_dict['<unk>'] = torch.randn((1, d_model))

In [7]:
sentence_odyssey = 'I must go back to my ship and to my crew'
sentence_amazon = 'We process and ship your order'
# in the most cost-efficient way possible'
words_o = sentence_odyssey.lower().split()
words_a = sentence_amazon.lower().split()

In [8]:
def embedding_matrix(words, embeddings_dict):
    X = [embeddings_dict[word] for word in words]
    X = torch.stack(X)
    return X

In [9]:
X = embedding_matrix(words_o, embeddings_dict)

In [10]:
X.size()

torch.Size([11, 50])

In [11]:
X_batch = torch.unsqueeze(X, dim=0)
X_batch

tensor([[[ 1.1891e-01,  1.5255e-01, -8.2073e-02, -7.4144e-01,  7.5917e-01,
          -4.8328e-01, -3.1009e-01,  5.1476e-01, -9.8708e-01,  6.1757e-04,
          -1.5043e-01,  8.3770e-01, -1.0797e+00, -5.1460e-01,  1.3188e+00,
           6.2007e-01,  1.3779e-01,  4.7108e-01, -7.2874e-02, -7.2675e-01,
          -7.4116e-01,  7.5263e-01,  8.8180e-01,  2.9561e-01,  1.3548e+00,
          -2.5701e+00, -1.3523e+00,  4.5880e-01,  1.0068e+00, -1.1856e+00,
           3.4737e+00,  7.7898e-01, -7.2929e-01,  2.5102e-01, -2.6156e-01,
          -3.4684e-01,  5.5841e-01,  7.5098e-01,  4.9830e-01, -2.6823e-01,
          -2.7443e-03, -1.8298e-02, -2.8096e-01,  5.5318e-01,  3.7706e-02,
           1.8555e-01, -1.5025e-01, -5.7512e-01, -2.6671e-01,  9.2121e-01],
         [ 4.7769e-01, -1.2242e-01,  2.9476e-01, -5.2751e-01,  6.9156e-01,
          -1.0851e-01,  2.0465e-01,  2.9798e-01, -2.5924e-02, -2.3163e-01,
           4.9905e-01,  7.5481e-01, -1.5678e-01,  2.6700e-02,  6.6682e-01,
           9.9944e-01,  

In [12]:
X_batch.size()

torch.Size([1, 11, 50])

$$
\text{Attention}({Q}, {K}, {V}) = \text{softmax}\left (\frac{{Q}  {K}^\intercal}{\sqrt{d_k}} \right)  {V},
$$

In [13]:
def attention(Q, K, V):
    d_k = K.size(dim=-1)
    scale = d_k ** -0.5
    attn_weights = F.softmax(Q @ K.T*scale, dim=-1)
    attn_output = attn_weights @ V
    return attn_output, attn_weights

In [14]:
attn_output, attn_weights = attention(X, X, X)

In [15]:
attn_output.size()

torch.Size([11, 50])

In [16]:
attn_output[6]

tensor([ 1.0387,  0.1033,  0.3426, -0.4320,  0.2237, -0.0958, -0.9926,  0.6662,
         0.4424, -0.7942,  0.5638,  0.9921,  0.0205,  0.5082,  0.0743, -0.1773,
        -0.3408,  0.5675, -1.1545, -0.5718,  0.4288,  0.4191, -0.0658, -0.3339,
         0.6682, -1.7473, -0.0485,  0.1531,  0.8642, -0.1447,  2.6571, -0.0545,
        -0.5343,  0.3160,  0.4041,  0.2277,  0.3958, -0.2916, -0.1126, -0.1385,
         0.1744, -0.5375,  0.9499, -0.4145, -0.1039, -0.1755, -0.2213, -0.3995,
         0.2119, -0.3610])

In [17]:
attn_weights[6]

tensor([0.0303, 0.0302, 0.0276, 0.0407, 0.0459, 0.0343, 0.5530, 0.0297, 0.0459,
        0.0343, 0.1281])

## Attention with Projections

In [18]:
class Attention(nn.Module):
    def __init__(self, d_model, d_k):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.scale = d_k ** -0.5
        self.WQ = nn.Linear(d_model, d_k, bias=False)
        self.WK = nn.Linear(d_model, d_k, bias=False)
        self.WV = nn.Linear(d_model, d_k, bias=False)

    def forward(self, X):
        Q = self.WQ(X)
        K = self.WK(X)
        V = self.WV(X)
        attn_weights = F.softmax(Q @ K.T*self.scale, dim=-1)
        attn_output = attn_weights @ V
        return attn_output, attn_weights

In [19]:
d_model = embeddings_dict[next(iter(embeddings_dict))].size(dim=-1)
d_model

50

In [20]:
attn = Attention(d_model, d_model)

In [21]:
att_out, att_weights = attn(X)

In [22]:
att_out

tensor([[-1.6623e-01, -3.2289e-01, -3.6165e-01, -2.7903e-02, -2.3599e-01,
          4.7430e-01,  6.5428e-01, -2.8029e-01,  2.0756e-02,  5.3674e-01,
         -8.3914e-02, -5.4726e-01, -5.4179e-01,  2.7358e-01,  8.2042e-01,
         -8.8333e-02,  5.8626e-02,  3.1333e-01, -4.4335e-01,  4.0806e-01,
          3.1551e-01, -4.1586e-02, -1.5990e-01,  1.9130e-01,  4.0484e-01,
         -2.3194e-01,  1.1097e-01,  3.9249e-02, -2.8538e-01,  4.4958e-02,
         -1.1391e-01,  5.1877e-01, -3.1351e-02,  1.8618e-01,  3.7673e-01,
         -3.7783e-01,  3.2962e-01, -3.5203e-01,  2.1866e-01,  2.1663e-01,
          3.3362e-01, -5.1468e-01,  1.7154e-01, -1.7607e-02,  5.5630e-01,
          2.4584e-01,  5.0733e-01, -2.8888e-01,  2.9656e-01, -3.4202e-01],
        [-1.7108e-01, -3.1969e-01, -3.5901e-01, -2.8647e-02, -2.3346e-01,
          4.7101e-01,  6.5450e-01, -2.7812e-01,  1.8074e-02,  5.3323e-01,
         -8.2419e-02, -5.4471e-01, -5.4338e-01,  2.7221e-01,  8.2259e-01,
         -8.9572e-02,  6.3969e-02,  3

In [23]:
att_weights

tensor([[0.0744, 0.0772, 0.0814, 0.0876, 0.0891, 0.0753, 0.1438, 0.0880, 0.0891,
         0.0753, 0.1188],
        [0.0679, 0.0837, 0.0800, 0.0914, 0.0888, 0.0765, 0.1463, 0.0789, 0.0888,
         0.0765, 0.1212],
        [0.0720, 0.0822, 0.0837, 0.0865, 0.0892, 0.0759, 0.1357, 0.0897, 0.0892,
         0.0759, 0.1200],
        [0.0658, 0.0815, 0.0808, 0.0829, 0.0909, 0.0712, 0.1542, 0.0881, 0.0909,
         0.0712, 0.1225],
        [0.0664, 0.0807, 0.0794, 0.0909, 0.0885, 0.0735, 0.1530, 0.0829, 0.0885,
         0.0735, 0.1227],
        [0.0695, 0.0817, 0.0827, 0.0829, 0.0912, 0.0694, 0.1543, 0.0862, 0.0912,
         0.0694, 0.1215],
        [0.0678, 0.0957, 0.0899, 0.0936, 0.0926, 0.0819, 0.1105, 0.0959, 0.0926,
         0.0819, 0.0976],
        [0.0759, 0.0850, 0.0813, 0.0867, 0.0875, 0.0786, 0.1410, 0.0834, 0.0875,
         0.0786, 0.1144],
        [0.0664, 0.0807, 0.0794, 0.0909, 0.0885, 0.0735, 0.1530, 0.0829, 0.0885,
         0.0735, 0.1227],
        [0.0695, 0.0817, 0.0827, 0.08

## Multihead attention

In [24]:
class MultiheadAttention(nn.Module):
    def __init__(self, d_model, h):
        super().__init__()
        self.h = h
        d_k = d_model // h
        self.attn_modules = nn.ModuleList(
            [Attention(d_model, d_k) for _ in range(h)])
        self.WO = nn.Linear(d_model, d_model, bias=False)

    def forward(self, X):
        attn_heads, attn_weights = zip(
            *[attn_module(X) for attn_module in self.attn_modules])
        attn_output = self.WO(torch.cat(attn_heads, dim=-1))
        attn_weights = torch.sum(torch.stack(attn_weights),
                                 dim=0)/self.h
        return attn_output, attn_weights

In [25]:
h = 5
multihead_attn = MultiheadAttention(d_model, h)

In [26]:
attn_output, attn_weights = multihead_attn(X)

In [27]:
attn_output.size(), attn_weights.size()

(torch.Size([11, 50]), torch.Size([11, 11]))

In [28]:
attn_output

tensor([[-0.0476,  0.0971, -0.2834, -0.0025, -0.0784, -0.1255,  0.0030,  0.0129,
          0.1317,  0.1839, -0.1862, -0.2681,  0.1798,  0.1930, -0.1245,  0.0827,
          0.0704, -0.2206, -0.2917,  0.1680,  0.0738,  0.0976, -0.3515,  0.1676,
         -0.1140, -0.2064, -0.0919, -0.0226, -0.4319, -0.5557, -0.2912, -0.0844,
         -0.0881, -0.2669,  0.1640,  0.1006,  0.1010,  0.0087,  0.4225, -0.3989,
          0.4763,  0.0778,  0.1797,  0.0377, -0.1116, -0.1889,  0.1885, -0.1090,
          0.0822, -0.1300],
        [-0.0422,  0.0979, -0.2805,  0.0017, -0.0836, -0.1249, -0.0054,  0.0156,
          0.1330,  0.1845, -0.1814, -0.2664,  0.1776,  0.1950, -0.1214,  0.0706,
          0.0782, -0.2158, -0.2916,  0.1679,  0.0771,  0.0986, -0.3488,  0.1726,
         -0.1181, -0.2064, -0.0880, -0.0173, -0.4313, -0.5478, -0.3017, -0.0807,
         -0.0915, -0.2621,  0.1648,  0.0938,  0.0985,  0.0124,  0.4275, -0.4023,
          0.4690,  0.0771,  0.1849,  0.0387, -0.1051, -0.1829,  0.1903, -0.1136,


In [29]:
attn_weights

tensor([[0.0946, 0.0909, 0.0970, 0.0954, 0.0897, 0.0946, 0.0802, 0.0899, 0.0897,
         0.0946, 0.0834],
        [0.0928, 0.0950, 0.0961, 0.0919, 0.0939, 0.0876, 0.0869, 0.0895, 0.0939,
         0.0876, 0.0849],
        [0.0931, 0.0907, 0.0953, 0.0927, 0.0906, 0.0939, 0.0853, 0.0891, 0.0906,
         0.0939, 0.0850],
        [0.0930, 0.0895, 0.0937, 0.0923, 0.0905, 0.0946, 0.0849, 0.0906, 0.0905,
         0.0946, 0.0858],
        [0.0924, 0.0929, 0.0942, 0.0929, 0.0927, 0.0899, 0.0876, 0.0885, 0.0927,
         0.0899, 0.0862],
        [0.0941, 0.0895, 0.0940, 0.0971, 0.0895, 0.0941, 0.0831, 0.0893, 0.0895,
         0.0941, 0.0859],
        [0.0940, 0.0895, 0.0922, 0.0868, 0.0860, 0.0974, 0.0869, 0.0904, 0.0860,
         0.0974, 0.0933],
        [0.0938, 0.0904, 0.0940, 0.0958, 0.0915, 0.0935, 0.0850, 0.0875, 0.0915,
         0.0935, 0.0834],
        [0.0924, 0.0929, 0.0942, 0.0929, 0.0927, 0.0899, 0.0876, 0.0885, 0.0927,
         0.0899, 0.0862],
        [0.0941, 0.0895, 0.0940, 0.09

## Residual networks

In [30]:
class LayerNormAttention(nn.Module):
    def __init__(self, d_model, h):
        super().__init__()
        self.multihead_attn = MultiheadAttention(d_model, h)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, X):
        Y, _ = self.multihead_attn(X)
        return self.layer_norm(X + Y)

In [31]:
ln_m_attn = LayerNormAttention(d_model, h)

In [32]:
ln_m_attn(X).size()

torch.Size([11, 50])

## Encoder Layer

In [33]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, h, d_ff=2048):
        super().__init__()

        self.multihead_attn = MultiheadAttention(d_model, h)
        self.layer_norm_1 = nn.LayerNorm(d_model)
        self.W1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.W2 = nn.Linear(d_ff, d_model)
        self.layer_norm_2 = nn.LayerNorm(d_model)

    def forward(self, X):
        Xprime, _ = self.multihead_attn(X)
        Xprime = self.layer_norm_1(X + Xprime)

        Y = self.W2(self.relu(self.W1(Xprime)))
        Y = self.layer_norm_2(Xprime + Y)
        return Y

In [34]:
encoder_layer = TransformerEncoderLayer(d_model,
                                        h,
                                        d_ff=4*d_model)

In [35]:
encoder_layer(X)

tensor([[ 4.0748e-01,  8.9986e-02, -6.5798e-01, -8.3133e-01,  1.1374e+00,
         -1.2454e+00, -6.2132e-01,  8.2200e-01, -1.6020e+00,  5.4709e-01,
         -4.8716e-01,  5.7372e-01, -1.4139e+00, -3.9440e-01,  1.7053e+00,
          4.3618e-01,  9.0066e-01,  7.1039e-01, -3.7073e-01, -9.0682e-01,
         -7.0589e-01,  1.0841e+00,  5.6641e-01,  5.6650e-01,  5.8410e-01,
         -3.1851e+00, -1.3922e+00,  5.6050e-01,  5.7870e-01, -1.0569e+00,
          3.5019e+00,  5.3748e-01, -6.0415e-01,  1.6018e-01, -5.2298e-01,
         -8.1777e-01,  1.0807e+00,  2.3649e-01,  4.0062e-01, -5.1705e-01,
         -3.4633e-01,  3.4833e-01, -2.4533e-01,  2.9764e-01,  2.0694e-01,
          2.9945e-01, -6.6185e-01, -4.5022e-01, -9.1076e-02,  7.8769e-01],
        [ 7.6640e-01, -2.3102e-01, -7.8056e-02, -8.4530e-01,  1.2035e+00,
         -8.9743e-01,  4.1055e-02,  5.3061e-01, -5.3280e-01,  3.2248e-02,
          4.2630e-01,  8.0076e-01, -5.8117e-01,  2.2586e-01,  1.0713e+00,
          9.4250e-01,  1.6072e+00, -4

## Encoder

In [36]:
encoder_layer = TransformerEncoderLayer(d_model, h, d_ff=4*d_model)
num_layers = 6

In [37]:
class TransformerEncoder(nn.Module):
    def __init__(self,
                 encoder_layer,
                 num_layers):
        super().__init__()
        self.encoder_stack = nn.ModuleList(
            [copy.deepcopy(encoder_layer)
             for _ in range(num_layers)])

    def forward(self, x):
        for encoder_layer in self.encoder_stack:
            x = encoder_layer(x)
        return x

In [38]:
encoder = TransformerEncoder(encoder_layer, num_layers)

In [39]:
encoder(X)

tensor([[ 1.0311, -0.9620, -0.6337,  1.2508,  1.2786, -0.9828, -2.6312, -0.2033,
         -0.1441, -1.7762, -1.2139,  0.5345,  1.4864, -1.6671,  1.1969,  0.0965,
          0.6430, -0.3987, -0.1126, -1.1541, -0.3055,  0.0868,  1.4157, -0.2065,
          0.1501,  1.0265,  0.4144, -1.0718,  0.2196, -0.0057,  2.1044,  1.6623,
         -0.1819,  0.8515, -0.2333, -0.6555, -0.9429,  0.8106, -0.3245,  1.5128,
          1.0559, -0.2312, -0.4746,  0.0057,  0.0900,  0.2692, -0.3647, -2.1271,
          0.1072, -0.2954],
        [ 1.0933, -0.9864, -0.7037,  1.1900,  1.0830, -0.3622, -2.9496, -0.3929,
         -0.1659, -1.5739, -1.4621,  0.4122,  1.4537, -1.2162,  0.7704, -0.4409,
          1.0764, -0.2389,  0.5925, -1.0907, -0.4872, -0.1929,  1.2132, -0.2454,
         -0.1177,  1.1209,  0.8869, -1.0616, -0.2661, -0.1117,  2.5177,  1.6885,
         -0.5112,  0.3641, -0.3021, -0.1987, -0.4189, -0.0560, -0.4552,  1.8628,
          0.9980,  0.1774, -0.1444, -0.1752,  0.0794,  0.4125, -0.5309, -1.8933,


## Input Embeddings

In [40]:
embeddings_words = embeddings_dict.keys()
print('Words in GloVe:', len(embeddings_dict.keys()))

Words in GloVe: 400000


In [41]:
embeddings_dict

{'the': tensor([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
         -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
          2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
          1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
         -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
         -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
          4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
          7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
         -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
          1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01]),
 ',': tensor([ 0.0134,  0.2368, -0.1690,  0.4095,  0.6381,  0.4771, -0.4285, -0.5564,
         -0.3640, -0.2394,  0.1300, -0.0637, -0.3957, -0.4816,  0.2329,  0.0902,
         -0.1332,  0.0786, -0.4163, -0.1543,  0.1007,  0.4889,  0.3123, -0.1252,
   

The vocabulary consists of the GloVe words. We could add them the words in the corpus

In [42]:
vocabulary = embeddings_dict.keys()

In [43]:
idx2word = dict(enumerate(vocabulary, start=2))
word2idx = {word: idx for idx, word in idx2word.items()}

In [44]:
input_embeddings = torch.rand(
    (len(word2idx) + 2, d_model))/10 - 0.05  # range: -0.05, 0.05

In [45]:
for word in embeddings_dict:
    # If the words are in the embeddings, we fill them with a value
    input_embeddings[word2idx[word]] = embeddings_dict[word]

## Positional Encodings

In [46]:
def pos_encoding(idx, d_model):
    angles = [idx / (10000 ** (2 * j / d_model)) for j in range(d_model)]
    pe = [math.sin(angle) if i % 2 == 0
          else math.cos(angle) for i, angle in enumerate(angles)]
    return len(angles), pe

In [47]:
pos_encoding(4, 50)

(50,
 [-0.7568024953079283,
  -0.9307752026113622,
  0.9415062110105988,
  0.24378998857303116,
  0.7933832927126037,
  0.8056897785422777,
  0.42466449969116443,
  0.9543169293275685,
  0.20838457529672513,
  0.989472469606138,
  0.10030648729934574,
  0.9975850112785689,
  0.048072042985881594,
  0.9994465862752576,
  0.023015565055894787,
  0.999873211223926,
  0.011016691956451144,
  0.9999709538962321,
  0.005273002518337972,
  0.9999933458972107,
  0.0025238266985760983,
  0.9999984756318129,
  0.001207980394376433,
  0.9999996507873545,
  0.0005781758760855534,
  0.999999920000001,
  0.00027673238483550924,
  0.9999999816730588,
  0.00013245244820575305,
  0.9999999958015403,
  6.33957276559798e-05,
  0.9999999990381885,
  3.0343102996511175e-05,
  0.9999999997796617,
  1.45231221902935e-05,
  0.9999999999495234,
  6.951203314941526e-06,
  0.9999999999884365,
  3.327055084404545e-06,
  0.9999999999973509,
  1.5924286822133147e-06,
  0.9999999999993932,
  7.621842871852255e-07,
 

In [48]:
"""def pos_encoding_1(seq_len, d_model):
    angles = torch.zeros((seq_len, d_model))
    for i in range(seq_len):
        for j in range(0, d_model, 2):
            angle = torch.tensor(i / (10000.0 ** (j / d_model)))
            angles[i, j] = torch.sin(angle)
            angles[i, j + 1] = torch.cos(angle)
    return angles"""

'def pos_encoding_1(seq_len, d_model):\n    angles = torch.zeros((seq_len, d_model))\n    for i in range(seq_len):\n        for j in range(0, d_model, 2):\n            angle = torch.tensor(i / (10000.0 ** (j / d_model)))\n            angles[i, j] = torch.sin(angle)\n            angles[i, j + 1] = torch.cos(angle)\n    return angles'

In [49]:
def pos_encoding(max_len, d_model):
    dividend = torch.arange(max_len).unsqueeze(0).T
    divisor = torch.pow(10000.0,
                        torch.arange(0, d_model, 2)/d_model)
    angles = dividend / divisor
    pe = torch.zeros((max_len, d_model))
    pe[:, 0::2] = torch.sin(angles)
    pe[:, 1::2] = torch.cos(angles)
    return pe

In [50]:
pos_embeddings = pos_encoding(100, d_model)

In [51]:
pos_embeddings

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  6.3795e-01,  ...,  1.0000e+00,
          1.4454e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.8254e-01,  ...,  1.0000e+00,
          2.8909e-04,  1.0000e+00],
        ...,
        [ 3.7961e-01, -9.2515e-01, -9.0618e-01,  ...,  9.9979e-01,
          1.4020e-02,  9.9990e-01],
        [-5.7338e-01, -8.1929e-01, -9.6761e-01,  ...,  9.9979e-01,
          1.4165e-02,  9.9990e-01],
        [-9.9921e-01,  3.9821e-02, -5.8410e-01,  ...,  9.9979e-01,
          1.4309e-02,  9.9990e-01]])

In [52]:
class Embedding(nn.Module):
    def __init__(self,
                 vocab_size,
                 d_model,
                 dropout=0.1,
                 max_len=500):
        super().__init__()
        self.d_model = d_model
        self.input_embedding = nn.Embedding(vocab_size, d_model)
        pe = self.pos_encoding(max_len, d_model)
        self.pos_embedding = nn.Embedding.from_pretrained(
            pe, freeze=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, X):
        pos_mat = torch.arange(X.size(dim=-1))
        X = self.input_embedding(X) * self.d_model**0.5
        X += self.pos_embedding(pos_mat)
        return self.dropout(X)

    def pos_encoding(self, max_len, d_model):
        dividend = torch.arange(max_len).unsqueeze(0).T
        divisor = torch.pow(10000.0,
                            torch.arange(0, d_model, 2)/d_model)
        angles = dividend / divisor
        pe = torch.zeros((max_len, d_model))
        pe[:, 0::2] = torch.sin(angles)
        pe[:, 1::2] = torch.cos(angles)
        return pe

In [53]:
vocab_size = len(word2idx) + 2
vocab_size

400002

In [54]:
embedding = Embedding(vocab_size, d_model)

In [55]:
embedding.input_embedding.weight

Parameter containing:
tensor([[-3.6916e-01,  2.1348e-01,  4.2756e-02,  ..., -2.3690e-01,
         -5.9199e-01,  1.4093e-03],
        [ 9.3115e-01,  2.3508e-02, -1.5762e-01,  ...,  8.1754e-01,
          1.2239e-01, -1.1047e+00],
        [ 2.6571e-01, -7.6492e-02, -8.0957e-01,  ...,  2.4078e+00,
          7.9436e-01, -9.5996e-01],
        ...,
        [ 1.7415e+00,  1.2242e+00, -7.5482e-01,  ...,  1.0991e+00,
          8.9916e-01,  2.8310e-01],
        [ 6.9783e-01,  8.8784e-02, -1.4850e+00,  ...,  1.1769e+00,
          6.2709e-01,  3.6045e-01],
        [ 5.7365e-02,  5.5242e-01,  9.7888e-02,  ...,  1.3633e-01,
          1.2499e-01, -1.7476e-01]], requires_grad=True)

In [56]:
embedding.input_embedding.weight = nn.Parameter(input_embeddings)

In [57]:
input_embeddings

tensor([[ 4.6659e-02,  2.5867e-02, -1.2617e-03,  ..., -2.2183e-02,
          1.6327e-02, -3.8216e-02],
        [ 7.9739e-03, -4.1976e-03,  3.7377e-02,  ..., -2.6118e-02,
          1.2064e-02, -2.2579e-02],
        [ 4.1800e-01,  2.4968e-01, -4.1242e-01,  ..., -1.8411e-01,
         -1.1514e-01, -7.8581e-01],
        ...,
        [-5.1181e-01,  5.8706e-02,  1.0913e+00,  ..., -2.5003e-01,
         -1.1250e+00,  1.5863e+00],
        [-7.5898e-01, -4.7426e-01,  4.7370e-01,  ...,  7.8954e-01,
         -1.4116e-02,  6.4480e-01],
        [ 7.2617e-02, -5.1393e-01,  4.7280e-01,  ..., -1.8907e-01,
         -5.9021e-01,  5.5559e-01]])

In [58]:
embedding.input_embedding.weight

Parameter containing:
tensor([[ 4.6659e-02,  2.5867e-02, -1.2617e-03,  ..., -2.2183e-02,
          1.6327e-02, -3.8216e-02],
        [ 7.9739e-03, -4.1976e-03,  3.7377e-02,  ..., -2.6118e-02,
          1.2064e-02, -2.2579e-02],
        [ 4.1800e-01,  2.4968e-01, -4.1242e-01,  ..., -1.8411e-01,
         -1.1514e-01, -7.8581e-01],
        ...,
        [-5.1181e-01,  5.8706e-02,  1.0913e+00,  ..., -2.5003e-01,
         -1.1250e+00,  1.5863e+00],
        [-7.5898e-01, -4.7426e-01,  4.7370e-01,  ...,  7.8954e-01,
         -1.4116e-02,  6.4480e-01],
        [ 7.2617e-02, -5.1393e-01,  4.7280e-01,  ..., -1.8907e-01,
         -5.9021e-01,  5.5559e-01]], requires_grad=True)

In [59]:
words_o

['i', 'must', 'go', 'back', 'to', 'my', 'ship', 'and', 'to', 'my', 'crew']

In [60]:
x = torch.LongTensor(
    list(map(lambda x: word2idx.get(x, 1), words_o)))

In [61]:
x

tensor([  43,  392,  244,  139,    6,  194, 1372,    7,    6,  194, 1696])

In [62]:
x.size()

torch.Size([11])

In [63]:
embeddings_dict['must']

tensor([ 0.4777, -0.1224,  0.2948, -0.5275,  0.6916, -0.1085,  0.2046,  0.2980,
        -0.0259, -0.2316,  0.4990,  0.7548, -0.1568,  0.0267,  0.6668,  0.9994,
         0.7260, -0.2042,  0.7568, -0.9753, -0.1006, -0.1912,  0.3810,  0.2040,
         0.1715, -1.8234, -0.1860, -0.4276,  0.3640, -0.5013,  3.5515,  0.6970,
        -1.3702, -0.6317, -0.0866, -0.1688,  0.4567, -0.0492, -0.1802, -0.4342,
        -0.2246, -0.4825,  0.5571,  0.4735, -0.2184,  0.1794, -0.2613,  0.3251,
        -0.1078,  0.1224])

In [64]:
pos_embeddings[1]

tensor([8.4147e-01, 5.4030e-01, 6.3795e-01, 7.7008e-01, 4.6056e-01, 8.8763e-01,
        3.2511e-01, 9.4568e-01, 2.2709e-01, 9.7387e-01, 1.5783e-01, 9.8747e-01,
        1.0943e-01, 9.9399e-01, 7.5785e-02, 9.9712e-01, 5.2457e-02, 9.9862e-01,
        3.6300e-02, 9.9934e-01, 2.5116e-02, 9.9968e-01, 1.7377e-02, 9.9985e-01,
        1.2022e-02, 9.9993e-01, 8.3175e-03, 9.9997e-01, 5.7544e-03, 9.9998e-01,
        3.9811e-03, 9.9999e-01, 2.7542e-03, 1.0000e+00, 1.9055e-03, 1.0000e+00,
        1.3183e-03, 1.0000e+00, 9.1201e-04, 1.0000e+00, 6.3096e-04, 1.0000e+00,
        4.3652e-04, 1.0000e+00, 3.0200e-04, 1.0000e+00, 2.0893e-04, 1.0000e+00,
        1.4454e-04, 1.0000e+00])

In [65]:
embedding.eval()

Embedding(
  (input_embedding): Embedding(400002, 50)
  (pos_embedding): Embedding(500, 50)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [66]:
X = embedding(x)

In [67]:
X.size()

torch.Size([11, 50])

In [68]:
X

tensor([[ 8.4082e-01,  2.0787e+00, -5.8034e-01, -4.2428e+00,  5.3681e+00,
         -2.4173e+00, -2.1927e+00,  4.6399e+00, -6.9797e+00,  1.0044e+00,
         -1.0637e+00,  6.9234e+00, -7.6346e+00, -2.6388e+00,  9.3253e+00,
          5.3846e+00,  9.7432e-01,  4.3310e+00, -5.1530e-01, -4.1389e+00,
         -5.2408e+00,  6.3219e+00,  6.2353e+00,  3.0903e+00,  9.5799e+00,
         -1.7173e+01, -9.5622e+00,  4.2442e+00,  7.1192e+00, -7.3835e+00,
          2.4563e+01,  6.5082e+00, -5.1569e+00,  2.7750e+00, -1.8495e+00,
         -1.4525e+00,  3.9486e+00,  6.3102e+00,  3.5235e+00, -8.9667e-01,
         -1.9405e-02,  8.7061e-01, -1.9867e+00,  4.9116e+00,  2.6662e-01,
          2.3120e+00, -1.0624e+00, -3.0667e+00, -1.8859e+00,  7.5139e+00],
        [ 4.2192e+00, -3.2534e-01,  2.7222e+00, -2.9600e+00,  5.3506e+00,
          1.2035e-01,  1.7722e+00,  3.0527e+00,  4.3778e-02, -6.6400e-01,
          3.6866e+00,  6.3248e+00, -9.9917e-01,  1.1828e+00,  4.7909e+00,
          8.0642e+00,  5.1858e+00, -4

In [69]:
Y = encoder(X)

In [70]:
Y.size()

torch.Size([11, 50])

## Extensions
We add batches to our implementation of attention

In [71]:
sentences = [words_o] + [words_a]
sentences

[['i', 'must', 'go', 'back', 'to', 'my', 'ship', 'and', 'to', 'my', 'crew'],
 ['we', 'process', 'and', 'ship', 'your', 'order']]

In [72]:
sent_idx = []
for sent in sentences:
    sent_idx += [torch.LongTensor(
        list(map(lambda x: word2idx.get(x, 1),
                 sent)))]
sent_idx

[tensor([  43,  392,  244,  139,    6,  194, 1372,    7,    6,  194, 1696]),
 tensor([  55,  548,    7, 1372,  394,  462])]

In [73]:
from torch.nn.utils.rnn import pad_sequence
X_idx = pad_sequence(sent_idx, batch_first=True)

In [74]:
X_idx

tensor([[  43,  392,  244,  139,    6,  194, 1372,    7,    6,  194, 1696],
        [  55,  548,    7, 1372,  394,  462,    0,    0,    0,    0,    0]])

Now $X$ contains a batch of two sentences

In [75]:
X = embedding(X_idx)

In [76]:
X.size()

torch.Size([2, 11, 50])

This will crash as our encoder only accepts one sample

In [77]:
try:
    encoder(X)
except Exception as e:
    print(f"Error: {e}")

Error: The size of tensor a (2) must match the size of tensor b (10) at non-singleton dimension 0


/var/folders/29/s86dlfss3gq6ybhg4frthtxr0000gq/T/ipykernel_61856/969577941.py:15: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4483.)
  attn_weights = F.softmax(Q @ K.T*self.scale, dim=-1)


In [78]:
try:
    result = X @ X.T
except Exception as e:
    print(f"Error during matrix multiplication: {e}")

Error during matrix multiplication: The size of tensor a (2) must match the size of tensor b (50) at non-singleton dimension 0


In [79]:
X.size()

torch.Size([2, 11, 50])

In [80]:
X @ X.transpose(1, 2)

tensor([[[ 1.9776e+03,  1.2268e+03,  1.3743e+03,  1.2414e+03,  1.1272e+03,
           1.7483e+03,  6.1404e+02,  9.4549e+02,  1.1061e+03,  1.7224e+03,
           6.5901e+02],
         [ 1.2268e+03,  1.3440e+03,  1.1234e+03,  9.0696e+02,  1.1117e+03,
           1.1106e+03,  6.1380e+02,  8.2709e+02,  1.0992e+03,  1.0989e+03,
           5.9510e+02],
         [ 1.3743e+03,  1.1234e+03,  1.2943e+03,  1.0968e+03,  1.0492e+03,
           1.2631e+03,  5.8847e+02,  7.9584e+02,  1.0380e+03,  1.2535e+03,
           6.3917e+02],
         [ 1.2414e+03,  9.0696e+02,  1.0968e+03,  1.3372e+03,  1.0484e+03,
           1.1991e+03,  6.9111e+02,  8.9329e+02,  1.0291e+03,  1.1896e+03,
           6.6301e+02],
         [ 1.1272e+03,  1.1117e+03,  1.0492e+03,  1.0484e+03,  1.1913e+03,
           1.0327e+03,  7.2530e+02,  9.5604e+02,  1.1856e+03,  1.0304e+03,
           6.1298e+02],
         [ 1.7483e+03,  1.1106e+03,  1.2631e+03,  1.1991e+03,  1.0327e+03,
           1.9106e+03,  6.6007e+02,  8.9511e+02,  1.001

We can multiply tensors of order 3 in Pytorch and NumPy, but this is something different. See here https://docs.pytorch.org/docs/stable/generated/torch.tensordot.html and here https://numpy.org/doc/stable/reference/generated/numpy.tensordot.html

In [81]:
torch.tensordot(X, torch.transpose(X, 0, 1), dims=([1, 0], [0, 1]))

tensor([[428.9768, -37.8188,  92.5769,  ...,  19.1320,  39.5217, -56.3458],
        [-37.8188, 115.2247,  -9.1544,  ..., -89.9425,  25.4529,  63.4553],
        [ 92.5769,  -9.1544, 131.4800,  ..., -15.4874,  15.6768,   4.8874],
        ...,
        [ 19.1320, -89.9425, -15.4874,  ..., 189.5553, -13.9252,  27.2004],
        [ 39.5217,  25.4529,  15.6768,  ..., -13.9252,  51.3503, -20.3683],
        [-56.3458,  63.4553,   4.8874,  ...,  27.2004, -20.3683, 201.6551]],
       grad_fn=<ViewBackward0>)

## Improving our Attention Class

In [82]:
class MultiheadAttention(nn.Module):
    def __init__(self, d_model, h, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.h = h
        self.d_k = d_model // h
        self.scale = self.d_k ** -0.5
        assert self.d_k * h == d_model, "d_model must be divisible by h"

        self.WQ = nn.Linear(d_model, d_model, bias=False)
        self.WK = nn.Linear(d_model, d_model, bias=False)
        self.WV = nn.Linear(d_model, d_model, bias=False)
        self.WO = nn.Linear(d_model, d_model, bias=False)

        self.dropout = nn.Dropout(dropout)

    def forward(self, X, key_padding_mask=None):
        # X: (bs, n, d_model)
        if X.dim() == 2:
            X = X.unsqueeze(0)
        batch_size, n, _ = X.size()

        # (bs, n, d_model) -> (bs, n, h, d_k) -> (bs, h, n, d_k)
        Q = self.WQ(X).reshape(batch_size, n, self.h, self.d_k).transpose(1, 2)
        K = self.WK(X).reshape(batch_size, n, self.h, self.d_k).transpose(1, 2)
        V = self.WV(X).reshape(batch_size, n, self.h, self.d_k).transpose(1, 2)

        # Scores: (bs, h, n, n)
        attn_scores = Q @ K.transpose(-2, -1) * self.scale

        if key_padding_mask is not None:
            # Before size (bs, n) after: (bs, 1, 1, n)
            key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            attn_scores = attn_scores.masked_fill(
                key_padding_mask == True, float('-inf'))

        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = attn_weights @ V  # (bs, h, n, d_k)

        # (bs, h, n, d_k) -> (bs, n, nhead*d_k) = (bs, n, d_model)
        attn_output = attn_output.transpose(
            1, 2).reshape(batch_size, n, self.d_model)
        attn_output = self.WO(attn_output)
        attn_weights = attn_weights.mean(dim=1)  # (bs, n, n)
        attn_weights = self.dropout(attn_weights)

        # Squeeze if bs=1
        attn_output = attn_output.squeeze(
            0) if batch_size == 1 else attn_output
        attn_weights = attn_weights.squeeze(
            0) if batch_size == 1 else attn_weights

        return attn_output, attn_weights

In [83]:
X

tensor([[[ 0.8408,  2.0787, -0.5803,  ..., -3.0667, -1.8859,  7.5139],
         [ 4.2192, -0.3253,  2.7222,  ...,  3.2991, -0.7622,  1.8656],
         [ 1.9578,  0.8397,  3.9769,  ..., -0.5429,  0.9175,  3.3301],
         ...,
         [ 5.8010, -0.4231,  1.4539,  ...,  0.4817, -0.4563, -0.8416],
         [-1.5168,  4.5700, -0.7766,  ..., -4.1878,  0.2869,  2.8849],
         [ 6.2673, -1.0631,  5.3749,  ...,  1.1481, -1.2829,  1.3748]],

        [[ 4.0579, -1.3143,  0.4987,  ...,  4.4478, -1.3038,  5.9454],
         [ 5.6869, -2.8417, -3.2670,  ...,  6.1935, -0.6640,  0.9464],
         [ 2.8056,  0.5983, -0.9887,  ..., -3.4696, -1.7695, -1.6939],
         ...,
         [ 1.3193,  0.0374, -0.6895,  ...,  0.8431,  0.1166,  0.7298],
         [ 0.7420, -0.7282, -0.0656,  ...,  0.8431,  0.1167,  0.7298],
         [-0.2141, -0.6562,  0.5844,  ...,  0.8431,  0.1169,  0.7298]]],
       grad_fn=<AddBackward0>)

In [84]:
mha = MultiheadAttention(50, 5)

In [85]:
key_padding_mask = (X_idx == 0)

In [86]:
key_padding_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False],
        [False, False, False, False, False, False,  True,  True,  True,  True,
          True]])

In [87]:
mha(X, key_padding_mask=key_padding_mask)

(tensor([[[-2.5203,  0.8368, -3.7002,  ..., -1.7859,  0.0710, -1.4290],
          [-3.0010,  1.1735, -3.0263,  ..., -0.8783, -0.6984, -1.5456],
          [-2.1401,  1.2171, -3.8675,  ..., -0.4297,  0.7816, -0.3949],
          ...,
          [-0.9726,  1.2426, -3.2340,  ..., -1.1101,  0.5304, -0.7726],
          [-2.0493,  1.1534, -3.4893,  ..., -0.9176,  0.6015, -1.0517],
          [-2.7543,  0.1040, -3.0345,  ..., -0.8815, -0.0795, -1.8796]],
 
         [[-3.6777,  0.2524, -3.5246,  ..., -1.1481,  0.0913, -2.7440],
          [-3.0378,  1.2666, -3.1719,  ..., -1.7953, -0.0970, -3.8988],
          [-3.0566,  0.2530, -3.5316,  ..., -1.0370, -0.5799, -4.0377],
          ...,
          [-2.1005, -1.4746, -2.8159,  ..., -0.1266, -1.5109, -2.4780],
          [-2.1512, -1.0500, -2.9373,  ..., -0.4601, -1.4936, -2.6899],
          [-2.3514, -0.5283, -3.2894,  ..., -0.6969, -1.2605, -2.7358]]],
        grad_fn=<UnsafeViewBackward0>),
 tensor([[[1.7229e-01, 2.5207e-01, 5.9986e-02, 1.0033e-01, 6.

## PyTorch Modules

### Multihead Attention

In [88]:
X.size()

torch.Size([2, 11, 50])

In [89]:
key_padding_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False],
        [False, False, False, False, False, False,  True,  True,  True,  True,
          True]])

In [90]:
multihead_attn = nn.MultiheadAttention(d_model,
                                       h,
                                       batch_first=True)

In [91]:
X.size()

torch.Size([2, 11, 50])

In [92]:
attn_output, attn_weights = multihead_attn(X, X, X, 
                                           key_padding_mask=key_padding_mask)

In [93]:
attn_output.size(), attn_weights.size()

(torch.Size([2, 11, 50]), torch.Size([2, 11, 11]))

In [94]:
attn_output

tensor([[[-0.3272, -1.0660,  1.1017,  ...,  2.2148, -2.4116,  0.6025],
         [-1.1081, -0.4734,  0.7497,  ...,  1.5647, -1.7563,  0.9655],
         [-0.2918, -1.0981,  0.7365,  ...,  2.3120, -1.7968,  0.9732],
         ...,
         [-0.8833, -0.5141,  0.8783,  ...,  1.5259, -2.2206,  1.1148],
         [-0.7188, -1.1684,  1.0507,  ...,  2.5215, -2.0184,  0.4980],
         [ 1.0014, -1.4364,  0.6641,  ...,  3.4299, -1.1994,  0.5382]],

        [[ 1.1919, -1.2394, -1.4967,  ..., -0.0726, -2.3806,  0.3976],
         [ 0.5646, -1.2284, -0.7807,  ..., -0.4404, -3.8826, -0.3772],
         [ 0.4608, -1.1435, -0.4521,  ..., -1.1667, -3.2226,  1.1068],
         ...,
         [ 2.0753, -1.3999, -0.7379,  ...,  0.5475, -2.5977,  1.8585],
         [ 2.3249, -1.4978, -0.9241,  ...,  0.2509, -2.5322,  1.7044],
         [ 2.2729, -1.5207, -0.9595,  ...,  0.0377, -2.5802,  1.5570]]],
       grad_fn=<TransposeBackward0>)

In [95]:
multihead_attn.state_dict()

OrderedDict([('in_proj_weight',
              tensor([[ 0.1468, -0.0164, -0.0693,  ...,  0.1248, -0.0034,  0.1046],
                      [-0.0975,  0.0115,  0.1313,  ...,  0.1324, -0.0650,  0.0816],
                      [ 0.0111,  0.0042,  0.1066,  ...,  0.0985, -0.0223,  0.1131],
                      ...,
                      [ 0.0565,  0.0669, -0.1304,  ..., -0.1337,  0.0910, -0.0125],
                      [ 0.0181,  0.1348, -0.1382,  ..., -0.1332,  0.1418, -0.1322],
                      [-0.0140, -0.1573, -0.1559,  ...,  0.1026,  0.1215,  0.1077]])),
             ('in_proj_bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 

In [96]:
attn_weights

tensor([[[1.5303e-04, 1.9980e-01, 2.1459e-04, 2.5505e-01, 2.4965e-03,
          5.7090e-02, 2.4175e-01, 9.9892e-02, 7.8109e-04, 1.4277e-01,
          1.0570e-05],
         [9.8542e-04, 1.8524e-02, 1.8605e-01, 2.8555e-01, 4.5907e-03,
          6.3708e-02, 2.6505e-01, 5.1729e-02, 3.1728e-03, 1.2064e-01,
          1.0115e-05],
         [2.1627e-05, 1.8132e-01, 1.9513e-02, 3.4351e-01, 2.2303e-02,
          6.1774e-02, 2.6294e-01, 2.3024e-02, 1.0690e-02, 7.4362e-02,
          5.4242e-04],
         [7.8763e-03, 3.7705e-01, 8.2507e-03, 3.4176e-02, 1.0505e-02,
          2.6020e-02, 2.8803e-01, 2.1027e-01, 1.1968e-02, 2.5304e-02,
          5.4277e-04],
         [1.7820e-04, 6.9533e-02, 1.3257e-01, 1.6538e-01, 6.9569e-04,
          6.5219e-02, 3.6945e-01, 6.1415e-02, 6.7499e-04, 1.3489e-01,
          2.2229e-07],
         [4.3458e-05, 2.0003e-01, 4.0335e-05, 3.3861e-01, 4.5775e-04,
          5.6481e-02, 2.0038e-01, 6.0275e-02, 2.3594e-04, 1.4258e-01,
          8.6482e-04],
         [2.0003e-01, 

### Encoders

In [97]:
encoder_layer = nn.TransformerEncoderLayer(d_model,
                                           h,
                                           dim_feedforward=4*d_model,
                                           batch_first=True)

In [98]:
encoder = nn.TransformerEncoder(encoder_layer, num_layers)

/var/folders/29/s86dlfss3gq6ybhg4frthtxr0000gq/T/ipykernel_61856/569992820.py:1: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  encoder = nn.TransformerEncoder(encoder_layer, num_layers)


In [99]:
enc_layer_output = encoder_layer(X,
                                 src_key_padding_mask=key_padding_mask)

In [100]:
enc_layer_output.size()

torch.Size([2, 11, 50])

In [101]:
enc_output = encoder(X,
                     src_key_padding_mask=key_padding_mask)

In [102]:
enc_output.size()

torch.Size([2, 11, 50])

In [103]:
enc_output

tensor([[[-0.0183,  0.2664,  1.1951,  ..., -1.1824,  0.4820, -1.5780],
         [-0.1484, -0.1888,  1.3105,  ..., -0.9623,  0.2612, -2.2087],
         [ 0.3320,  0.0481,  1.1538,  ..., -0.2996,  0.3537, -1.6847],
         ...,
         [ 0.4778,  0.3239,  1.0586,  ..., -0.3177,  0.4296, -1.7853],
         [ 0.0296,  0.4870,  1.3719,  ..., -0.5901, -0.0955, -1.5827],
         [ 0.6747, -0.0763,  0.9794,  ..., -0.0902,  0.3573, -1.8089]],

        [[ 0.4800, -0.1433,  1.5437,  ...,  0.0474,  0.6508, -2.2200],
         [-0.3535, -0.1046,  1.5793,  ...,  0.3124,  0.5223, -2.3741],
         [ 0.1495,  0.2227,  1.0339,  ...,  0.2262, -0.3045, -2.6559],
         ...,
         [ 0.1901,  0.2101,  1.0321,  ...,  0.4549,  0.6230, -2.6338],
         [ 0.5193, -0.0709,  0.9958,  ...,  0.5837,  0.0323, -2.7810],
         [-0.0472,  0.3758,  1.0516,  ...,  0.3374,  0.4555, -2.5324]]],
       grad_fn=<NativeLayerNormBackward0>)